# Dataset Plotting

Plot multiple variables from an xarray Dataset with automatic or custom slot assignment.

In [ ]:
import numpy as np
import xarray as xr

from xarray_plotly import config, xpx

config.notebook()

In [ ]:
# Create a Dataset with multiple variables
time = np.arange(50)
cities = ["NYC", "LA", "Chicago"]

ds = xr.Dataset(
    {
        "temperature": (["time", "city"], 20 + 5 * np.random.randn(50, 3).cumsum(axis=0) / 10),
        "humidity": (["time", "city"], 50 + 10 * np.random.randn(50, 3).cumsum(axis=0) / 10),
        "pressure": (["time", "city"], 1013 + np.random.randn(50, 3).cumsum(axis=0)),
    },
    coords={"time": time, "city": cities},
)
ds

## Plot All Variables

When you call a plot method on a Dataset without specifying `var`, all variables are combined into a single DataArray with a new `"variable"` dimension:

In [ ]:
# All variables: time -> x, variable -> color, city -> line_dash
xpx(ds).line()

## Control Where "variable" Goes

The `"variable"` dimension can be assigned to any slot:

In [ ]:
# Variables as facet columns
xpx(ds).line(facet_col="variable")

In [ ]:
# Variables as rows, cities as columns
xpx(ds).line(facet_row="variable", facet_col="city")

## Configure Default "variable" Position

By default, `"variable"` is placed as the **second** dimension so it maps to `color`. This keeps your first dimension (e.g., time) on the x-axis.

You can change this globally with `config.set_options()`:

In [ ]:
# Default: position=1 (second) -> variable goes to color
# Note: to_array() puts "variable" first, but xpx() reorders it to position 1
print("Raw to_array() dims:", ds.to_array().dims)  # (variable, time, city)
print("After xpx reorder:   (time, variable, city)")  # time->x, variable->color
xpx(ds).line(title="Default: variable as color (position=1)")

In [ ]:
# Position 0: variable goes first (x-axis) - usually not what you want!
with config.set_options(dataset_variable_position=0):
    fig = xpx(ds).line(title="position=0: variable on x-axis (probably not desired)")
fig

In [ ]:
# Position -1: variable goes last -> city gets color, variable gets line_dash
with config.set_options(dataset_variable_position=-1):
    fig = xpx(ds).line(title="position=-1: variable as line_dash")
fig

## Plot a Single Variable

Use `var="name"` to plot just one variable:

In [ ]:
xpx(ds).line(var="temperature", title="Temperature Only")

## Different Plot Types

In [ ]:
# Bar chart - latest values by city
xpx(ds.isel(time=-1)).bar(x="city", color="variable", barmode="group")

In [ ]:
# Box plot - distribution by variable
xpx(ds).box(x="variable", color="city")

In [ ]:
# Area chart
xpx(ds).area(var="humidity", title="Humidity Over Time")

In [ ]:
# Scatter
xpx(ds).scatter(var="temperature", title="Temperature Scatter")

In [ ]:
# Pie chart - snapshot at one time
xpx(ds.isel(time=-1)).pie(var="temperature", names="city", title="Temperature Distribution")

## Combining Slot Assignments

Mix explicit assignments with auto-assignment:

In [ ]:
# Explicit: variable -> facet_col, let city auto-assign to color
xpx(ds).line(facet_col="variable", color="city")

In [ ]:
# Skip color slot with None
xpx(ds).line(var="temperature", color=None)